# Create csv file with Chexbert medical annotaions

In [1]:
!pip install transformers -q
!pip install sentencepiece -q

     |████████████████████████████████| 3.1 MB 29.1 MB/s 
     |████████████████████████████████| 59 kB 6.1 MB/s 
     |████████████████████████████████| 3.3 MB 44.2 MB/s 
     |████████████████████████████████| 895 kB 19.2 MB/s 
     |████████████████████████████████| 596 kB 38.5 MB/s 
     |████████████████████████████████| 1.2 MB 29.1 MB/s 


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Add the chexbert medical annotations in one column

In [4]:
#load chexbert annotations file
#this file was created through guideness of - https://github.com/stanfordmlgroup/CheXbert
chexbert_df = pd.read_csv("/content/drive/My Drive/Final project - Zebra/Final data/mimic-cxr-2.0.0-chexbert.csv")
chexbert_df.head()

,Report Impression,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,IMPRESSION: No acute cardiopulmonary process.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,IMPRESSION: No acute cardiopulmonary abnorm...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,IMPRESSION: No acute intrathoracic process.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,IMPRESSION: No acute cardiopulmonary process.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,IMPRESSION: Focal consolidation at the left...,NaN,NaN,NaN,NaN,NaN,1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#create one column of all medical tags in one string
chexbert_df = chexbert_df.replace([1.0,0.0,-1.0],['positive','negative','uncertain'])

for (columnName, columnData) in chexbert_df.iteritems():
  chexbert_df[columnName] = np.where((chexbert_df[columnName] == 'positive'),columnName+' positive',chexbert_df[columnName])
  chexbert_df[columnName] = np.where((chexbert_df[columnName] == 'negative'),columnName+' negative',chexbert_df[columnName])
  chexbert_df[columnName] = np.where((chexbert_df[columnName] == 'uncertain'),columnName+' uncertain',chexbert_df[columnName])

chexbert_df = chexbert_df.replace(['nan'],[np.nan])

chexbert_df['chexbert medical tags'] = chexbert_df[chexbert_df.columns[1:15]].apply(
    lambda x: ' ,'.join(x.dropna().astype(str)),
    axis=1
)

chexbert_df.drop(chexbert_df.columns[0:15], axis = 1, inplace = True)

chexbert_df.head()

,chexbert medical tags
0,No Finding positive
1,No Finding positive
2,No Finding positive
3,No Finding positive
4,"Consolidation positive ,Pneumonia uncertain"


In [8]:
#add the created column to the main csv file
tags_reports_df = pd.read_csv("/content/drive/My Drive/Final project - Zebra/Final data/chexpert medical tags and reports.csv")
tags_reports_df['chexbert medical tags'] = chexbert_df['chexbert medical tags']
#tags_reports_df.rename(columns={"medical tags": "chexpert medical tags"}, inplace = True)

from google.colab import drive
from google.colab import files
tags_reports_df.to_csv('chexbert&pert reports.csv', index = False)
files.download("chexbert&pert reports.csv")

tags_reports_df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,path,medical tags,report_impression,report_findings,start_report,chexbert medical tags
0,files/p10/p10000032/s50414267.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,"FINDINGS: There is no focal consolidation, ...",FINAL REPORT ...,No Finding positive
1,files/p10/p10000032/s53189527.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary abnorm...,"FINDINGS: The cardiac, mediastinal and hila...",FINAL REPORT ...,No Finding positive
2,files/p10/p10000032/s53911762.txt,No Finding positive,IMPRESSION: No acute intrathoracic process.,FINDINGS: Single frontal view of the chest ...,FINAL REPORT ...,No Finding positive
3,files/p10/p10000032/s56699142.txt,No Finding positive,IMPRESSION: No acute cardiopulmonary process.,FINDINGS: The lungs are clear of focal cons...,FINAL REPORT ...,No Finding positive
4,files/p10/p10000764/s57375967.txt,"Consolidation positive ,Pneumonia uncertain",IMPRESSION: Focal consolidation at the left...,FINDINGS: PA and lateral views of the chest...,FINAL REPORT ...,"Consolidation positive ,Pneumonia uncertain"
